In [1]:
import os
import sys
import logging
import time

import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pprint import pprint

import supertrend as spt
from ftx_client import FtxClient
from config import API_KEY, API_SECRET
from telegram_api_manager import TelegramAPIManager

plt.ioff()

In [2]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
testing = False

In [ ]:
tapi = TelegramAPIManager(group=False)

INTERVAL = "4h"
idx = 1

while(True):
    # Open new FTX Session
    ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

    if idx == 1 or idx % 12 == 0:
        perpetuals = []
        for future in ftx.list_futures():
            if future["type"] == "perpetual":
                if future["volume"] >  10e6:
                    perpetuals.append(future["name"])
    
    idx += 1
        
    for perp in perpetuals:
        df = ftx.get_historical_market_data(perp, interval=INTERVAL, start_time="40 days ago")

        # Perform supertrend analysis
        df["st"], df["upt"], df["dt"] = spt.supertrend_analysis(df.high, df.low, df.close, look_back=10, multiplier=3)
        df["long_trig"], df["short_trig"], df["st_signal"] = spt.get_supertrend_signals(df.close, df.st)
        
        # Check 200 EMA for confirmation
        df["ema200"] = spt.calculate_ema(df.close, time_period=200)
        df["vol_ema200"] = spt.calculate_ema(df.volume, time_period=200)   
        
        figure_path = spt.plot_and_save_figure(perp, df, folder=INTERVAL)
        
        # Set precision for orders
        precision = len(str(df.close[0]).split(".")[1])
        
        # Take profit calculator and stop loss
        stop_loss = round(df.st[-1], precision)
        long_profit_10pct, short_profit_10pct = spt.take_profit_calc(df.close[-1], profit_percent=10, precision=precision)
        long_profit_5pct, short_profit_5pct = spt.take_profit_calc(df.close[-1], profit_percent=5, precision=precision)
        
        # Check last element of signal array
        last_signal = df.st_signal[-1]
        
        if testing:
            last_signal = -1
            
        if last_signal != 0:
            # Set up dict for new position
            new_position = {
                "market": perp,
                "interval": INTERVAL,
                "entry": df.close[-1], 
                "stop_loss": stop_loss,
            }
            
            if last_signal == 1:
                # Long position
                new_position["side"] = "buy"
                new_position["10pctprofit"] = long_profit_10pct
                new_position["5pctprofit"] = long_profit_5pct
            elif last_signal == -1:
                # Short position
                new_position["side"] = "sell"
                new_position["10pctprofit"] = short_profit_10pct
                new_position["5pctprofit"] = short_profit_5pct

            if df.close[-1] < df.ema200[-1]:
                new_position["ema200"] = "under"
            else:
                new_position["ema200"] = "over"
            
            # Use slowd for StochRSI
            _, slowd = spt.calculate_stoch_rsi(df)
            new_position["stoch_rsi"] = int(slowd[-1])
            
            logger.info(new_position)
            tapi.send_photo(figure_path, caption=json.dumps(new_position, indent=2, default=str))
            
            # Close position if needed
            open_position = ftx.check_open_position(perp)
            if open_position:
                if open_position["side"] != new_position["side"]:
                    success, response = market_close_and_cancel_orders(
                                            perp, 
                                            side=new_position["side"], 
                                            size=open_position["size"]
                                        )
                    
                    if success:
                        close_position_text = f"({perp}) Position closed at {response['price']}"
                    else:
                        close_position_text = f"({perp}) Position failed to close: Message {response}"
                    
                    tapi.send_photo(figure_path, caption=close_position_text)
            if testing:
                break
        else:

    logger.info("Sleeping for 4 hours")
    ftx._session.close()
    time.sleep(60 * 60 * 4)

2021-07-18 00:29:14,849 - telegram_api_manager - INFO - cryptobot (@pymparoobot) is now running using Telegram personal chat id
2021-07-18 00:30:09,637 - __main__ - INFO - Sleeping for 4 hours
2021-07-18 04:31:02,992 - __main__ - INFO - Sleeping for 4 hours
2021-07-18 08:31:55,515 - __main__ - INFO - Sleeping for 4 hours
2021-07-18 12:32:50,600 - __main__ - INFO - Sleeping for 4 hours
2021-07-18 16:33:42,921 - __main__ - INFO - Sleeping for 4 hours
2021-07-18 20:34:42,621 - __main__ - INFO - Sleeping for 4 hours
